In [1]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver


import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini", openai_api_key=api_key)

conn = sqlite3.connect("memory.db", check_same_thread=False)

config = {
    "configurable": {
        "thread_id": "1",
    }
}

In [2]:
class State(MessagesState):
    pass


graph_builder = StateGraph(State)

In [3]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response],
    }

In [4]:
graph_builder.add_node("chatbot", chatbot)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [ ]:
result = graph.invoke(
    {
        "messages": [
            {"role": "user", "content": ""},
        ],
    },
    config=config,
)

In [8]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hello my name is Jeongmok.
================================== Ai Message ==================================

Hello, Jeongmok! How can I assist you today?
================================ Human Message =================================

I live in hwaseong-si
================================== Ai Message ==================================

That’s great! Hwaseong-si is known for its beautiful sights and rich history. Do you have any specific questions or topics you’d like to discuss about your city or anything else?


In [13]:
state_history = graph.get_state_history(config)

In [14]:
for state_snapshot in list(state_history):
    print(state_snapshot.next)
    print(state_snapshot.values["messages"])
    print("===================\n")

()
[HumanMessage(content='Hello my name is Jeongmok.', additional_kwargs={}, response_metadata={}, id='add87475-c33e-49b3-80ce-a8fb6cf10f59'), AIMessage(content='Hello, Jeongmok! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CPlVpJGXIwhDuakBSDL4PKz3ItOIs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e38cf984-6487-47df-9c9e-8774bc3b3c19-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage

In [17]:
state_history = graph.get_state_history(config)

to_fork = list(state_history)[-5]

to_fork.values["messages"]

[HumanMessage(content='Hello my name is Jeongmok.', additional_kwargs={}, response_metadata={}, id='add87475-c33e-49b3-80ce-a8fb6cf10f59'),
 AIMessage(content='Hello, Jeongmok! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CPlVpJGXIwhDuakBSDL4PKz3ItOIs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e38cf984-6487-47df-9c9e-8774bc3b3c19-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(

In [19]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages": HumanMessage(
            content="I live in seoul-si",
            id="ece006c2-99a1-4d59-81fd-b3721d3fdbed",
        )
    },
)

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0a747a-6fee-66ad-8004-fc0113dd6251'}}

In [21]:
forked_state = graph.get_state_history(
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_ns": "",
            "checkpoint_id": "1f0a747a-6fee-66ad-8004-fc0113dd6251",
        }
    }
)

list(forked_state)[-1].values["messages"]

[HumanMessage(content='Hello my name is Jeongmok.', additional_kwargs={}, response_metadata={}, id='add87475-c33e-49b3-80ce-a8fb6cf10f59'),
 AIMessage(content='Hello, Jeongmok! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CPlVpJGXIwhDuakBSDL4PKz3ItOIs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e38cf984-6487-47df-9c9e-8774bc3b3c19-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(

In [22]:
result = graph.invoke(
    None,
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_ns": "",
            "checkpoint_id": "1f0a747a-6fee-66ad-8004-fc0113dd6251",
        }
    },
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hello my name is Jeongmok.
================================== Ai Message ==================================

Hello, Jeongmok! How can I assist you today?
================================ Human Message =================================

I live in seoul-si
================================== Ai Message ==================================

That's great! Seoul-si is a vibrant city with a rich history and culture. Is there something specific you’d like to talk about or ask regarding Seoul or anything else?
